# Preprocess Landsat and MODIS Data

This notebook preprocesses datasets from Landsat and MODIS, respectively. Data is from Google Earth Engine, processed, and stored as TFRecord in Google Drive, through the following steps:

1. Load city data

Only US cities are included at this moment. The data is from the 2020 US Census Bureau's [TIGER/Line Shapefiles](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html). 

2. Prepare Landsat data

Data ranges from 2003-present.

Landsat 7: collection 2 level 2 tier 1 surface reflectance data.
(Try using both two consecutive months and two same months from consecutive years.)

SLC gap fill:
Apply the USGS L7 Phase-2 Gap filling protocol, using a single kernel size. 

3. Load Modis data

Data ranges from 2003-present.

Reproject Modis to Landsat projection.

Aqua data.LST_Day_1km.

4. Download Data in TFRecord Format to Google Drive

In [40]:
######################################################## Import libraries ######################################################
import ee
import geemap
import sys
import os
import tensorflow as tf
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
from dateutil.relativedelta import *
from gee_prep import filter_collection, rescale_rename_landsat, export_to_drive, get_gap_filled_image, preprocess_modis
from gee_prep import preprocess_modis

In [2]:
np.set_printoptions(threshold=sys.maxsize)

In [3]:
######################################################### Define variables #####################################################
drive_folder = 'Data_TFRecord_Daily'
landsat_collection = 'LANDSAT/LE07/C02/T1_L2'
modis_collection = 'MODIS/061/MYD11A1'
us_city_collection = 'users/yiwenz9/us_cities'
city_name, city_num = ['Phoenix', 'Phoenix--Mesa, AZ']
predictors = ['Blue','Green','Red','NIR','SWIR1','SWIR2','ST']
#Preserve only the qa bands of most important images used to fill gaps.
qa_bands = ['source_mask', 'qa_pixel_0', 'qa_radsat_0','qa_pixel_1', 'qa_radsat_1', 'qa_pixel_2', 'qa_radsat_2']
target = 'LST_Day_1km'
landsat_res = 30 #m
modis_res = 1000 #m
radius = 16 #(33-1)/2 pixels

In [4]:
######################################################### Initiate a map #######################################################
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
cities = ee.FeatureCollection(us_city_collection)
target_city=cities.filter(ee.Filter.eq('NAME10',city_num))
Map.addLayer(target_city,{}, city_name)

In [6]:
start_date = datetime.date(2010, 12, 1)
end_date = datetime.date(2019, 3, 1)
delta = relativedelta(months=3)

while start_date < end_date:
    start = str(start_date)
    end = str(start_date + delta)
    print(start, end)
    landsat = filter_collection(landsat_collection, start_date=start, end_date=end,geometry=target_city)\
            .map(rescale_rename_landsat).sort('CLOUD_COVER')
    gap_filled_imgCol=get_gap_filled_image(landsat)
    img_export=gap_filled_imgCol.select(predictors+qa_bands).median().reproject(landsat.first().projection()).unmask(-9999)
    modis_reproj=preprocess_modis(modis_collection, start_date=start, end_date=end,geometry=target_city)
    modis_reproj_list = modis_reproj.toList(modis_reproj.size())
    for i in range(modis_reproj.size().getInfo()):
        modis_download = ee.Image(modis_reproj_list.get(i))
        vector=modis_download.sample(
          region= target_city.geometry(),
          dropNulls=False,
          geometries=True,
        )
        fname=ee.Date(modis_download.get('system:time_start')).format("YYMMdd").getInfo()
#        print(fname)
        export_to_drive(img = img_export, region = vector, radius = radius,
                        units = 'pixels', scale = landsat_res, folder = drive_folder, fname = fname)
    start_date += delta

2010-12-01 2011-03-01
2011-03-01 2011-06-01
2011-06-01 2011-09-01
2011-09-01 2011-12-01
2011-12-01 2012-03-01
2012-03-01 2012-06-01
2012-06-01 2012-09-01
2012-09-01 2012-12-01
2012-12-01 2013-03-01
2013-03-01 2013-06-01
2013-06-01 2013-09-01
2013-09-01 2013-12-01
2013-12-01 2014-03-01
2014-03-01 2014-06-01
2014-06-01 2014-09-01
2014-09-01 2014-12-01
2014-12-01 2015-03-01
2015-03-01 2015-06-01
2015-06-01 2015-09-01
2015-09-01 2015-12-01
2015-12-01 2016-03-01
2016-03-01 2016-06-01
2016-06-01 2016-09-01
2016-09-01 2016-12-01
2016-12-01 2017-03-01
2017-03-01 2017-06-01
2017-06-01 2017-09-01
2017-09-01 2017-12-01
2017-12-01 2018-03-01
2018-03-01 2018-06-01
2018-06-01 2018-09-01
2018-09-01 2018-12-01
2018-12-01 2019-03-01


# Read a TFRecord file

In [26]:
from tfrecords2numpy import TFRecordsParser

In [9]:
dataset=tf.data.TFRecordDataset(
    '060306.tfrecord.gz', compression_type='GZIP', buffer_size=None, num_parallel_reads=None
)

In [29]:
record=TFRecordsParser('060306.tfrecord').tfrecrods2numpy()

In [ ]:
record

In [50]:
len(record)

2969

In [30]:
from tqdm import tqdm

In [ ]:
for idx, (features, lst) in tqdm(enumerate(record), desc="Storing Files", position=1, total=len(record), leave=False):
    print(lst)

In [10]:
bands=['Blue','Green','Red','NIR','SWIR1','SWIR2', 'ST', 'source_mask', 'qa_pixel_0', 'qa_radsat_0','qa_pixel_1', 'qa_radsat_1']
bands=['Blue','Green','Red']
feature_names=list(bands)
feature_names

['Blue', 'Green', 'Red']

In [16]:
target='LST_Day_1km'

In [17]:
# List of fixed-length features, all of which are float32.
predictor_columns = [
  tf.io.FixedLenFeature(shape=[33,33], dtype=tf.float32) for k in feature_names
]
target_columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(feature_names+[target], predictor_columns+target_columns))

pprint(features_dict)

{'Blue': FixedLenFeature(shape=[33, 33], dtype=tf.float32, default_value=None),
 'Green': FixedLenFeature(shape=[33, 33], dtype=tf.float32, default_value=None),
 'LST_Day_1km': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'Red': FixedLenFeature(shape=[33, 33], dtype=tf.float32, default_value=None)}


In [18]:
#Count the number of examples in a TFRecord
dataset.reduce(np.int64(0), lambda x, _: x + 1)

<tf.Tensor: shape=(), dtype=int64, numpy=2969>

In [19]:
def input_fn(serialized_example):

 # Make a parsing function
    example = tf.io.parse_single_example(serialized_example,features_dict)
    return example
  
  # Passing of FeatureColumns to a 4D tensor
#     def stack_images(features):         
#         nfeat = tf.transpose(tf.squeeze(tf.stack(list(features.values()))))
#         return nfeat
  
#     dataset = dataset.map(parse)
#     dataset = dataset.map(stack_images)
  
#     if shuffle:
#         dataset = dataset.shuffle(buffer_size = batchSize * 10)
#     dataset = dataset.batch(batchSize)
#     dataset = dataset.repeat(numEpochs)
parsed_dataset= dataset.map(input_fn)

In [ ]:
for elements in parsed_dataset.take(1):
    print(elements)

In [ ]:
#rescale to 255
display_num = 10
plt.figure(figsize=(9, 40))

c=0
for i in range(5, display_num):
    for x in parsed_dataset.take(i):
        x
    Red=x['Red'].numpy().clip(0,1)
#    Red=(Red-Red.min())/(Red.max()-Red.min())
    Green=x['Green'].numpy().clip(0,1)
#    Green=(Green-Green.min())/(Green.max()-Green.min())
    Blue=x['Blue'].numpy().clip(0,1)
#    Blue=(Blue-Blue.min())/(Blue.max()-Blue.min())
    LST=x[target].numpy()[0]
#    tensor = tf.squeeze(x).numpy()
#   #print(target.sum())  
    plt.subplot(display_num, 2, c + 1)
    plt.imshow(np.stack((Red,Green,Blue),-1))
    plt.title(f'LST:{LST}')
    c+=2 
plt.savefig('TFRecord_vis')
plt.show()